In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mlds

Mounted at /content/drive
/content/drive/MyDrive/mlds


In [2]:
import data_auto_sys as aus
import tasker_link_prediction as t_lp
from splitter import splitter
from models import Baseline_node2vec
from trainer import Trainer
import yaml
import os

In [3]:
prep = False

with open('config/config_AS_baseline.yaml', 'r') as file:
    config = yaml.safe_load(file)

model_path = config['model_path']

if not os.path.exists(model_path):
    os.mkdir(model_path)
    os.mkdir(model_path + "best_checkpoints/")
    os.mkdir(model_path + "latest_checkpoints/")

with open(model_path + 'config.yaml', 'w') as file:
    yaml.dump(config, file)

config

{'adam_config': {'adam_beta_1': 0.9,
  'adam_beta_2': 0.999,
  'adam_epsilon': 1e-07,
  'adam_learning_rate': 0.01},
 'model_path': 'models/AS_N2V_baseline/',
 'classifier_hidden_size': 20,
 'ffn_fusion_size': 10,
 'ffn_hidden_size': 10,
 'gcn_fusion_size': 20,
 'spatial_hidden_size': 20,
 'spatial_input_dim': 128,
 'temporal_hidden_size': 20,
 'temporal_input_dim': 200,
 'num_epochs': 1000,
 'patience': 10,
 'major_threshold': None,
 'train_proportion': 0.7,
 'dev_proportion': 0.1,
 'data_path': 'data/AS-733/',
 'prep_data_path': 'prep_data/AS733_neg/'}

In [4]:
data = aus.Autonomous_System_Dataset(config['data_path'])
tasker = t_lp.Link_Pred_Tasker(data, path=config['prep_data_path'], prep=prep,
                               embs_dim=config['spatial_input_dim'], temp_dim=int(config['temporal_input_dim']/10),
                               major_threshold=config['major_threshold'], smart_neg_sampling=True)

splitter_as = splitter(tasker, train_proportion = config['train_proportion'], dev_proportion = config['dev_proportion'])

model= Baseline_node2vec(config['spatial_input_dim'])

trainer = Trainer(model=model, splitter=splitter_as, model_path=model_path, adam_config=config['adam_config'], patience=config['patience'])

Dataset splits sizes:  train 60 dev 10 test 20


In [5]:
trainer.train(config['num_epochs'])

Epoch 1, Step 1, Loss: 0.07661040127277374
Epoch 1, Step 2, Loss: 0.021808285266160965
Epoch 1, Step 3, Loss: 0.018378430977463722
Epoch 1, Step 4, Loss: 0.020400533452630043
Epoch 1, Step 5, Loss: 0.023163912817835808
Epoch 1, Step 6, Loss: 0.021660037338733673
Epoch 1, Step 7, Loss: 0.02187223546206951
Epoch 1, Step 8, Loss: 0.031005164608359337
Epoch 1, Step 9, Loss: 0.030089542269706726
Epoch 1, Step 10, Loss: 0.028898024931550026
Epoch 1, Step 11, Loss: 0.024003230035305023
Epoch 1, Step 12, Loss: 0.024607956409454346
Epoch 1, Step 13, Loss: 0.022434009239077568
Epoch 1, Step 14, Loss: 0.023440618067979813
Epoch 1, Step 15, Loss: 0.020527714863419533
Epoch 1, Step 16, Loss: 0.020121080800890923
Epoch 1, Step 17, Loss: 0.01828143186867237
Epoch 1, Step 18, Loss: 0.01830928772687912
Epoch 1, Step 19, Loss: 0.01859920471906662
Epoch 1, Step 20, Loss: 0.017412032932043076
Epoch 1, Step 21, Loss: 0.017159925773739815
Epoch 1, Step 22, Loss: 0.016236837953329086
Epoch 1, Step 23, Loss: 

In [6]:
trainer.restore_best_checkpoint()
trainer.validate('Validation')
trainer.validate('Test')

Validation Metric MRRs: 0.5970122737258886
Validation Metric MAPs: 0.4045650161204452
Test Metric MRRs: 0.5670882021358256
Test Metric MAPs: 0.3903435745433434


(0.5670882021358256, 0.3903435745433434)